<a href="https://colab.research.google.com/github/sayanbanerjee32/learning-equality-curriculum-recommendations/blob/main/lecr_SentTrans_data_prep_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle
## install collab specific libraries
if is_colab:
    !pip install -Uq datasets
    !pip install -Uq transformers
    !pip install -Uq sentencepiece 
    !pip install -Uq evaluate
    !pip install -Uq pynvml
!pip install -Uq sentence_transformers
!pip install -Uq faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.9 MB/s eta 0:00:00


In [3]:
# # This get the RAPIDS-Colab install files and test check your GPU.  Run cells 1 and 2 only.
# # Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/env-check.py

In [4]:
# # This will update the Colab environment and restart the kernel.
# !bash rapidsai-csp-utils/colab/update_gcc.sh
# import os
# os._exit(00)

In [5]:
# ## Installing CondaColab.  This will restart your kernel again
# import condacolab
# condacolab.install()

In [6]:
# # you can now run the rest of the cells as normal
# import condacolab
# condacolab.check()

In [7]:
# # Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# # The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
# # The <packages> option are default blank or 'core'.  By default, we install RAPIDSAI and BlazingSQL.  The 'core' option will install only RAPIDSAI and not include BlazingSQL, 
# !python rapidsai-csp-utils/colab/install_rapids.py nightly
# import os
# os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
# os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
# os.environ['CONDA_PREFIX'] = '/usr/local'

In [8]:
from fastkaggle import *
from pathlib import Path
import pandas as pd
import numpy as np # linear algebra
from tqdm.auto import tqdm
import gc

import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from datasets import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


In [45]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
comp = 'learning-equality-curriculum-recommendations'
if is_colab:
    model_save_path = Path('/content/'+comp+'_out/models')
    data_save_path = Path('/content/drive/MyDrive/Colab Notebooks/data/'+comp)
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')
    data_save_path = Path('/kaggle/working/'+comp)

In [10]:
# import colab libraries
if is_colab:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [12]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [13]:
path = setup_comp(comp)
path

100%|██████████| 254M/254M [00:02<00:00, 132MB/s]


Path('learning-equality-curriculum-recommendations')

### read all input files

In [14]:
# load the data into pandas dataframes
topics_df = pd.read_csv(path / "topics.csv", index_col=0).fillna({"title": "", "description": ""})
content_df = pd.read_csv(path / "content.csv", index_col=0).fillna("")
correlations_df = pd.read_csv(path / "correlations.csv", index_col=0)

In [15]:
print(topics_df.shape, correlations_df.shape, content_df.shape)
correlations_df.head()

(76972, 8) (61517, 1) (154047, 7)


,content_ids
topic_id,
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
t_00069b63a70a,c_11a1dc0bfb99
t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4


In [16]:
def process_correlations(correlations_df, label_value):
    correlations = correlations_df.copy()
    correlations.content_ids = correlations.content_ids.str.split()
    correlations = correlations.explode("content_ids").rename(columns={"content_ids": "content_id"}).reset_index()
    correlations['label'] = label_value
    return correlations
correlations = process_correlations(correlations_df, 1)
correlations.head()

,topic_id,content_id,label
0,t_00004da3a1b2,c_1108dd0c7a5d,1
1,t_00004da3a1b2,c_376c5a8eb028,1
2,t_00004da3a1b2,c_5bc0e1e2cba0,1
3,t_00004da3a1b2,c_76231f9d0b5e,1
4,t_00068291e9a4,c_639ea2ef9c95,1


In [17]:
content_df.head()

,title,description,kind,text,language,copyright_holder,license
id,,,,,,,
c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,,es,,
c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,,it,,
c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,,es,,
c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA


#### Negative Samples

In [18]:
### sample records for colab only - test bed
if is_colab:
    correlations_df = correlations_df.sample(frac=0.1)
    sampled_topic_ids = correlations_df.reset_index().topic_id.to_list()
    correlations = correlations.query('topic_id in @sampled_topic_ids')
    sampled_content_ids = correlations.content_id.to_list()
    tmp_content_df = content_df.query('id in @sampled_content_ids')
    negative_content_df = content_df.query('id not in @sampled_content_ids').sample(frac=0.1)
    content_df = pd.concat([tmp_content_df,negative_content_df], axis = 0)
correlations_df.shape, correlations.shape, content_df.shape

((6152, 1), (27882, 3), (37304, 7))

### Find all Similer contents for each topic

#### generate content embedding

In [19]:
# create data set
content_dataset = Dataset.from_pandas(content_df)

In [20]:
# TODO - need to find a way to use text
def concatenate_text(examples):
    return {
        "concat_text": (examples["title"]
        + " \n "
        + examples["description"].replace('\n',' ').strip()
        + " \n "
        + examples["text"].replace(examples["title"],'').strip().split("\n")[0][:100]
        ).strip()
    }
content_dataset = content_dataset.map(concatenate_text)

Map:   0%|          | 0/37304 [00:00<?, ? examples/s]

In [21]:
# model_nm = 'microsoft/mdeberta-v3-base'
# MAX_LEN = 256
model_nm = 'all-MiniLM-L6-v2'
MAX_LEN = 256
batch_size = 64
SELECT_TOP_N = 25

In [22]:
# Load models
# model = AutoModel.from_pretrained(model_nm)
# model.eval()
# model.to(device)

# tokenizer = AutoTokenizer.from_pretrained(model_nm)
## sentence transformer
model = SentenceTransformer(model_nm)

In [23]:
# helper function cleaning GPU memory
def report_gpu():
    if is_colab: print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [24]:
# helper funcion for embedding and pooling
# def cls_pooling(model_output):
#     return model_output.last_hidden_state[:, 0]

# def mean_pooling(model_output, attention_mask):
#     # Extract the token embeddings
#     token_embeddings = model_output[0]
#     # Compute the attention mask
#     input_mask_expanded = (attention_mask
#                            .unsqueeze(-1)
#                            .expand(token_embeddings.size())
#                            .float())
#     # Sum the embeddings, but ignore masked tokens
#     sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
#     sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#     # Return the average as a single vector
#     return sum_embeddings / sum_mask

# def get_embeddings(text_list):
#     encoded_input = tokenizer(
#         text_list, padding=True, truncation=True, return_tensors="pt",
#         max_length=MAX_LEN
#     )
#     encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
#     model_output = model(**encoded_input)
#     # pooled_embeds = mean_pooling(model_output, encoded_input["attention_mask"])
#     return cls_pooling(model_output)

In [25]:
# test embedding
# embedding = get_embeddings(content_dataset["concat_text"][0])
embedding = model.encode(content_dataset["concat_text"][0:2])
embedding.shape

(2, 384)

In [26]:
report_gpu()
content_embeddings_dataset = content_dataset.map( 
    lambda x: {"embeddings": (model.encode(x["concat_text"])
                                # .detach().cpu().numpy()[0]
                              )
                },
batched=True)

GPU:0
process       4347 uses     1126.000 MB GPU memory


Map:   0%|          | 0/37304 [00:00<?, ? examples/s]

In [27]:
report_gpu()
content_embeddings_dataset.add_faiss_index(column="embeddings")

GPU:0
process       4347 uses     1422.000 MB GPU memory


  0%|          | 0/38 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'description', 'kind', 'text', 'language', 'copyright_holder', 'license', 'id', 'concat_text', 'embeddings'],
    num_rows: 37304
})

#### generate topic embedding

In [28]:
topic_ids = correlations_df.reset_index()['topic_id'].tolist()
_topics_df = topics_df.query(f'id in {topic_ids}')
_topics_df.head()

,title,description,channel,category,level,language,parent,has_content
id,,,,,,,,
t_0010852b7049,Divisão com vários algarismos,You know your multiplication tables and are ge...,8e286a,source,4,pt,t_33355476f050,True
t_001bd0fea51b,Interest rates and the time value of money,"What is an interest rate, really? Its the paym...",0ec697,source,4,en,t_d2de1837a004,True
t_001edc523bd1,Algebraic equations basics,Explore what it means for equations with varia...,2ee29d,aligned,4,en,t_a6438798d90f,True
t_00269d6e7e45,"Kuzidisha kwa makumi, mamia, na maelfu",Jifunze kuzidisha namba nzima kwa vigawe vya k...,0c929f,source,4,sw,t_1c0c5d025888,True
t_0028ead4dc26,Types of profit,Economic profit and accounting profit are two ...,0ec697,source,4,en,t_8a1f2110da12,True


In [29]:
# create data set
topic_dataset = Dataset.from_pandas(_topics_df)
topic_dataset

Dataset({
    features: ['title', 'description', 'channel', 'category', 'level', 'language', 'parent', 'has_content', 'id'],
    num_rows: 6152
})

In [30]:
# concatenate title and description
def concatenate_topic_text(examples):
    return {
        "concat_text": (examples["title"]
        + " \n "
        + examples["description"].replace('\n',' ').strip()
        ).strip()
    }
topic_dataset = topic_dataset.map(concatenate_topic_text)

Map:   0%|          | 0/6152 [00:00<?, ? examples/s]

In [31]:
# test embedding
report_gpu()
# embedding = get_embeddings(topic_dataset["concat_text"][0])
embedding = model.encode(topic_dataset["concat_text"][0])
embedding.shape

GPU:0
process       4347 uses     1124.000 MB GPU memory


(384,)

In [32]:
report_gpu()
topic_embeddings_dataset = topic_dataset.map(
    # lambda x: {"embeddings": get_embeddings(x["concat_text"]).detach().cpu().numpy()[0]}
    lambda x: {"embeddings": model.encode(x["concat_text"])},
    batched=True
)

GPU:0
process       4347 uses     1126.000 MB GPU memory


Map:   0%|          | 0/6152 [00:00<?, ? examples/s]

### Similarity Search

In [33]:
## test with a single search
report_gpu()
scores, samples = content_embeddings_dataset.get_nearest_examples(
    "embeddings",
     np.array(topic_embeddings_dataset[0]['embeddings'], dtype=np.float32),
     k=SELECT_TOP_N
)
topic_embeddings_dataset[0]['concat_text'],samples['concat_text']

GPU:0
process       4347 uses     1422.000 MB GPU memory


('Divisão com vários algarismos \n You know your multiplication tables and are getting the hang of basic division. In this tutorial, we will journey into the world of loooong division (sometimes, referred to as "long division", but that\'s not as much fun to say). After this tutorial, you\'ll be able to divide any whole number by a single digit number.',
 ['Whole Number Division',
  'Multi-digit division \n Practice dividing multi-digit whole numbers. These problems use remainders.',
  'Basic multi-digit division \n Divide numbers like 105÷21 or 133÷14.',
  'Multiply and divide by 10   \n Practice multiplying and dividing whole numbers by ten.',
  'Divide whole numbers by 10 \n Practice dividing whole numbers by 10.',
  'Division of Decimals by Whole Numbers',
  'Dividing Whole Numbers by Decimals Practice',
  'How To Divide With Positive Whole Numbers  \n How To Divide With Positive Whole Numbers | Numbers | Maths | FuseSchool  This video will teach you how to apply different methods 

In [34]:
## using range search in order to get the indices
faiss_index = content_embeddings_dataset.get_index('embeddings').faiss_index
limits, distances, indices = faiss_index.range_search(x=np.array(topic_embeddings_dataset['embeddings'], dtype=np.float32).reshape(len(topic_embeddings_dataset),-1),thresh=0.97)
len(limits), len(indices), len(distances)

(6153, 1184696, 1184696)

In [35]:
neg_correlation_dict = {}
for i, j, t_id in zip(limits[:-1], limits[1:], topic_embeddings_dataset['id']):
    sorted_indices = [x for _, x in sorted(zip(distances[i:j], indices[i:j]))]
    neg_correlation_dict[t_id] = ' '.join([content_embeddings_dataset[int(ind)]['id'] for ind in sorted_indices[:SELECT_TOP_N]])

neg_correlation_df = pd.DataFrame(neg_correlation_dict.items(), columns = ['topic_id','content_ids']).set_index('topic_id')
print(neg_correlation_df.info(null_counts = True))
neg_correlation_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 6152 entries, t_0010852b7049 to t_ffeaa4905b1f
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   content_ids  6152 non-null   object
dtypes: object(1)
memory usage: 96.1+ KB
None


<ipython-input-35-db9d2222b6aa>:7: FutureWarning: null_counts is deprecated. Use show_counts instead
  print(neg_correlation_df.info(null_counts = True))


,content_ids
topic_id,
t_0010852b7049,c_5b536b8b15eb c_ddcf587856cb c_f7be6e887d9d c...
t_001bd0fea51b,c_6011b969f2ef c_e7a80cbd098a c_f106124090b3 c...
t_001edc523bd1,c_941db6dc7201 c_6af245b5ae64 c_3e957c9c6585 c...
t_00269d6e7e45,c_c525e71492af c_c8fcf05a92c0 c_f701a1046727 c...
t_0028ead4dc26,c_a4651c228714 c_e01965f1924d c_7a838ef49896 c...


In [36]:
neg_correlations = process_correlations(neg_correlation_df, 0)
neg_correlations.head()

,topic_id,content_id,label
0,t_0010852b7049,c_5b536b8b15eb,0
1,t_0010852b7049,c_ddcf587856cb,0
2,t_0010852b7049,c_f7be6e887d9d,0
3,t_0010852b7049,c_b5987a9c9790,0
4,t_0010852b7049,c_7d4e213eb64e,0


In [37]:
## contact positive and neagtive correlations
pos_neg_corr = pd.concat([correlations, neg_correlations])
#de-duplicate
pos_neg_corr = pos_neg_corr.sort_values(['topic_id','content_id','label'], ascending = [True, True, False]).drop_duplicates(['topic_id','content_id'])
pos_neg_corr.label.value_counts()

0    100125
1     27882
Name: label, dtype: int64

In [38]:
# ### batch search
# report_gpu()
# _, samples = content_embeddings_dataset.get_nearest_examples_batch(
#         "embeddings",
#         np.array(topic_embeddings_dataset['embeddings'], dtype=np.float32),
#         k=SELECT_TOP_N
#      )
# (len(samples),
#  topic_embeddings_dataset[1]['concat_text'],
#  samples[1]['concat_text'])

## Merge all DF

In [39]:
topic_content_details = pos_neg_corr.merge(topics_df,
                                           left_on = 'topic_id',
                                        right_on = 'id').merge(content_df,
                                                                  left_on = 'content_id',
                                                                right_on = 'id',
                                                               suffixes=('_topic', '_content'))
topic_content_details.head()

,topic_id,content_id,label,title_topic,description_topic,channel,category,level,language_topic,parent,has_content,title_content,description_content,kind,text,language_content,copyright_holder,license
0,t_0010852b7049,c_0baf72ed7e1e,1,Divisão com vários algarismos,You know your multiplication tables and are ge...,8e286a,source,4,pt,t_33355476f050,True,Introdução à divisão longa (sem restos),Assista a uma introdução à divisão longa com o...,video,"RKA - Nesse vídeo, vou apresentar um novo modo...",pt,Khan Academy,CC BY-NC-SA
1,t_0010852b7049,c_0e50263d2fcd,0,Divisão com vários algarismos,You know your multiplication tables and are ge...,8e286a,source,4,pt,t_33355476f050,True,Dividing decimals: hundredths,"Divide two numbers. Divisors, dividends, and ...",exercise,,en,,
2,t_6d9e96a6af2e,c_0e50263d2fcd,0,Dividing whole numbers,Learn long division on whole numbers. Solve pr...,0ec697,source,4,en,t_f08eeee1502c,True,Dividing decimals: hundredths,"Divide two numbers. Divisors, dividends, and ...",exercise,,en,,
3,t_ec7abcdc97e4,c_0e50263d2fcd,1,Dividing decimals,Learn long division with decimals. Solve probl...,0ec697,source,4,en,t_f08eeee1502c,True,Dividing decimals: hundredths,"Divide two numbers. Divisors, dividends, and ...",exercise,,en,,
4,t_0010852b7049,c_197191239da7,0,Divisão com vários algarismos,You know your multiplication tables and are ge...,8e286a,source,4,pt,t_33355476f050,True,Kugawanya kwa kutumia thamani ya namba,Decompose 3- and 4-digit dividends to divide t...,exercise,,sw,,


In [40]:
### Any case where has_content = False
sum(~topic_content_details.has_content)

0

In [41]:
## check thr proportion of positive and negative
topic_content_details.label.value_counts()

0    99950
1    27882
Name: label, dtype: int64

In [46]:
### dump final dataframe
data_save_path.mkdir(parents=True, exist_ok=True)
topic_content_details.to_pickle(data_save_path / 'topic_content_details.pkl')

In [43]:
if not iskaggle:
    push_notebook('saan', comp,
                  title='LECR: Sentence Transformer - Data prep - baseline',
                  file='/content/drive/MyDrive/Colab Notebooks/lecr_SentTrans_data_prep_baseline.ipynb',
                  competition=comp, private=False, gpu=True)

Your kernel title does not resolve to the specified id. This may result in surprising behavior. We suggest making your title something that resolves to the specified id. See https://en.wikipedia.org/wiki/Clean_URL#Slug for more information on how slugs are determined.


ApiException: ignored